Import

In [16]:
import pandas as pd
import os

Read league datas

In [17]:
# Create a list of leagues and their names in the excel files
leagues = ['premier_league', 'laliga', 'bundesliga', 'serie_a', 'ligue_1']

# Read excel files and combine them into a single dataframe
dfs = []
i = 0
# Loop through each league and read the corresponding excel file
for league in leagues:
    # Read the excel file
    df = pd.read_excel(f'../Datas/my_2020_{league}_data.xlsx')
    # Add a new column for the league name
    if i==0:
        df['league'] = 'Premier League'
    elif i==1:
        df['league'] = 'La Liga'
    elif i==2:
        df['league'] = 'Bundesliga'
    elif i==3:
        df['league'] = 'Serie A'
    elif i==4:
        df['league'] = 'Ligue 1'
    i += 1
    # Append the dataframe to the list
    dfs.append(df)

Process the data

In [18]:
# Concatenate all dataframes into a single dataframe
df = pd.concat(dfs, ignore_index=True)
# Drop the first row if it is empty
df.dropna(how='all', inplace=True)  
# Reset the index
df.reset_index(drop=True, inplace=True)    
# Drop every row except the name and value columns
df = df[['name','age', 'value','position','league']]  

# First, clean the value column but keep it as string initially
df['value'] = df['value'].str.replace('€', '').str.replace(',', '')

# Convert values with 'm' suffix to millions
df.loc[df['value'].str.contains('m', na=False), 'value'] = df['value'].loc[df['value'].str.contains('m', na=False)].str.replace('m', '').astype(float) * 1e6

# Convert values with 'k' suffix to thousands
df.loc[df['value'].str.contains('k', na=False), 'value'] = df['value'].loc[df['value'].str.contains('k', na=False)].str.replace('k', '').astype(float) * 1e3

# Convert remaining values to float
df.loc[~df['value'].str.contains('m|k', na=False), 'value'] = df.loc[~df['value'].str.contains('m|k', na=False), 'value'].astype(float)

# Sort in decreasing order of values
df.sort_values(by='value', ascending=False, inplace=True)
# Reset the index after sorting
df.reset_index(drop=True, inplace=True)

# Rename the columns to 'Player' and 'Value'
df.rename(columns={'name': 'Player','age':'Age','value': 'Value (2020)','position': 'Position','league':'League'}, inplace=True)

Save

In [19]:
print('Saving combined data to excel...')
output_file = '../Datas/combined_2020_data.xlsx'
if os.path.exists(output_file):
    os.remove(output_file)  # Remove the file if it already exists
df.to_excel(output_file, index=False)

Saving combined data to excel...
